In [2]:
import sys
import regex
import cgatcore.iotools as iotools
import pysam
import logging
import argparse

In [58]:
tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]


outfile = iotools.open_file("polya_revcomp.fastq", "w")
log =  iotools.open_file("revcomp" + ".log","w")
n = 0
y = 0
with pysam.FastxFile("input.fastq") as fh:
    
    for record in fh:
        y +=1
        if len(record.sequence) < 300:
            pass
        else:
            seq = record.sequence[30:300]
            m=regex.findall("(TTTTTTTTTTTTTTTTTTTT){e<=3}", str(seq))
            if m:
                n +=1
                sequence = reverse_complement_table(str(record.sequence))
                outfile.write("@%s\n%s\n+\n%s\n" % (record.name, sequence, record.quality))
            else:
                seq = record.sequence[-30:-200:]
                m=regex.findall("(AAAAAAAAAAAAAAAAAAAA){e<=3}", str(seq))
                if m:
                    n +=1
                    outfile.write("@%s\n%s\n+\n%s\n" % (record.name, record.sequence, record.quality))
                            
print(n)
print(y)
outfile.close()
log.close()

363
1000


In [89]:
tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]

def most_common(lst):
    return max(set(lst), key=lst.count)

outfile = open("polya_umi.fastq", "w")

n = 0

collapsed_trimers = []

with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:
        n += 1
        seq_nano = record.sequence
        
        m=regex.finditer("(GTACTCTGCGTTGATACCACTGCTT){e<=1}", str(record.sequence))
        
        for i in m:
            umi = seq_nano[i.start()-24:i.start()]
            #print(umi)
            single_nuc = []
            
            trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]
            
            for trimer in trimers:
                single_nuc.append(most_common(trimer))
            
            collapse = "".join(single_nuc)
            collapsed_trimers.append(collapse)

            
            
            
#print(collapsed_trimers)



In [95]:
# count the number of perfect 
from collections import Counter
counts = []
n = 0
for sequence in collapsed_trimers:
    n += 1
    count = sum(1 for a, b in zip("CTGCTACT", sequence) if a != b)
    #print(count)
    counts.append(count)
print(Counter(counts))

print("The percent of perfect UMIs with trimers: %s"%(Counter(counts)[0]/n*100))
print("The percent of one missmatch UMI with trimers: %s"%(Counter(counts)[1]/n*100))
print("The percent of two missmatch UMI with trimers: %s"%(Counter(counts)[2]/n*100))
print("The percent of three missmatch UMI with trimers: %s"%(Counter(counts)[3]/n*100))


Counter({0: 140, 2: 31, 1: 19, 3: 17, 5: 17, 4: 13, 8: 8, 7: 4, 6: 3})
The percent of perfect UMIs with trimers: 55.55555555555556
The percent of one missmatch UMI with trimers: 7.5396825396825395
The percent of two missmatch UMI with trimers: 12.3015873015873
The percent of three missmatch UMI with trimers: 6.746031746031746


In [119]:
# How many idels vs point mutations

def allCharactersSame(s) :
    n = len(s)
    for i in range(1, n) :
        if s[i] != s[0] :
            return False
 
    return True


indel = 0
point = 0
total = 0
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:
        total += 1
        seq_nano = record.sequence
        
        m=regex.finditer("(GTACTCTGCGTTGATACCACTGCTT){e<=1}", str(record.sequence))
        
        for i in m:
            umi = seq_nano[i.start()-24:i.start()]
            #print(umi)
            single_nuc = []
            
            trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]
            
            n = 0
            indel_counter = 0
            for trimer in trimers:
                
                if allCharactersSame(trimer):
                    if n > 1:
                        point += 1
                if not allCharactersSame(trimer):
                    #print(trimer)
                    n += 1
                    #print(n)
                    if n > 1:
                        indel_counter += 1
            if indel_counter > 1:
                indel += 1
print(total)
print(point)
print(indel)

print("Total UMIs that have either point or indels: {}".format((total-(point+indel))/total *100))
print("Total UMIs that have either point or indels: {}".format((point+indel)/total *100))
print("Total UMIs that have a point error: {}".format(point/total *100))
print("Total UMIs that have either indels error: {}".format(indel/total *100))
                        
        

363
138
218
Total UMIs that have either point or indels: 98.07162534435263
Total UMIs that have a point error: 38.01652892561984
Total UMIs that have either indels error: 60.05509641873278
